In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import requests

In [15]:
fer_datasets = ["CK+", "MMI", "JAFFE", "TFD", "FER-2013", "AFEW7.0", 
    "SFEW2.0", "Multi-PIE", "BU-3DFE", "Oulu-CASIA", 
    "RaFD", "KDEF", "EmotioNet", "RAF-DB", "AffectNet", "ExpW"]


In [16]:
fer_datasets_LONG = [
    "Extended Cohn-Kanade", 
    'MMI Facial Expression',
    "Japanese Female Facial Expression", 
    "Toronto Face Database",
    "Binghamton University 3D Facial Expression",
    "Oulu-CASIA",
    "Radboud Faces Database",
    "Karolinska Directed Emotional Faces",
    "Acted Facial Expressions In The Wild",
    "Static Facial Expression in the Wild",
    "CMU Multi-PIE",
    "Affective Faces Database",
    "Expression in-the-Wild"
]


In [17]:
topics = ["facial expression recognition", "FER", "deep learning", "machine learning", "classification", "classifier", "neural network", "CNN"]

In [18]:
file_path = '../Scrapes_ALL.csv'
Scrape_df = pd.read_csv(file_path)
Scrape_df.head()

,Unnamed: 0,Title,Authors,Year,Cited By,Detected_Dataset,Detected_Topic,Abstract,DOI,Journal,...,Mentions_Statistical_significance,Mentions_P-value,Mentions_T-test,Mentions_Anova,Mentions_Correlation,Mentions_Regression,Mentions_Baseline_comparison,Mentions_Mae,Mentions_Rmse,Mentions_Bias
0,0,10 Automated Face Analysis for Affective Compu...,"['JF Cohn', 'F De la Torre']",2015,170,Affective Faces Database,classifier,Differences in manual coding between databases...,No DOI,The Oxford handbook of affective …,...,False,False,False,False,False,False,False,False,False,False
1,1,3D facial expression recognition based on auto...,"['H Tang', 'TS Huang']",2008,205,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",facial expression recognition from 3D facial s...,No DOI,… on computer vision and pattern recognition …,...,False,False,False,False,False,False,False,False,False,False
2,2,3D facial expression recognition based on prim...,"['J Wang', 'L Yin', 'X Wei', 'Y Sun']",2006,440,Binghamton University 3D Facial Expression,facial expression recognition,expressions using 3D facial expression range d...,No DOI,… Vision and Pattern Recognition …,...,False,False,False,False,False,False,False,False,False,False
3,3,3D facial expression recognition based on prop...,"['H Tang', 'TS Huang']",2008,153,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",Binghamton University have recently constructe...,No DOI,… on Automatic Face & Gesture Recognition,...,False,False,False,False,False,False,False,False,False,False
4,4,3D facial expression recognition using SIFT de...,"['S Berretti', 'B Ben Amor', 'M Daoudi', 'A De...",2011,184,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",at the Binghamton University (BU-3DFE database...,No DOI,The Visual Computer,...,False,False,False,False,False,False,False,False,False,False


In [24]:
# Function to extract the base domain of a URL (until .com, .org, etc.)
def extract_base_domain(url):
    if pd.notna(url):
        # Split the URL and return the main domain (e.g., 'arxiv.org', 'ieee.org')
        return '.'.join(url.split('/')[2].split('.')[-2:])
    return None

# Apply the function to extract base domains
Scrape_df['Base_Domain'] = Scrape_df['URL'].apply(extract_base_domain)

# Get the counts of each base domain
base_domain_counts = Scrape_df['Base_Domain'].value_counts()

# Display the counts of base domains
print(base_domain_counts)


ieee.org              76
arxiv.org             37
sciencedirect.com     32
nih.gov               21
springer.com          15
acm.org               13
mdpi.com               6
thecvf.com             5
researchgate.net       4
paperswithcode.com     2
wiley.com              2
tandfonline.com        2
cmu.edu                1
oup.com                1
psu.edu                1
computer.org           1
kdef.se                1
auth.gr                1
mlr.press              1
nature.com             1
utpjournals.press      1
github.com             1
cedus.it               1
iop.org                1
sagepub.com            1
plos.org               1
toronto.edu            1
Name: Base_Domain, dtype: int64


# Handle arXiv Pages: 

The get_arxiv_pdf_link function fetches the HTML from the arXiv page and parses it with BeautifulSoup to find the "View PDF" link.

Download the PDF: Once the "View PDF" link is found, the script downloads the PDF as before.

File Naming and Saving: PDFs are saved with the name arxiv_paper_{index}.pdf in the pdfs directory.

In [26]:
'''from bs4 import BeautifulSoup


# Directory where you want to save the PDFs
output_dir = 'pdfs'
os.makedirs(output_dir, exist_ok=True)

# Function to get the direct PDF link from an arXiv page
def get_arxiv_pdf_link(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Look for the "View PDF" link
        pdf_link = soup.find('a', string='View PDF')
        if pdf_link:
            return 'https://arxiv.org' + pdf_link['href']
        else:
            print(f"PDF link not found on arXiv page: {url}")
            return None
    except Exception as e:
        print(f"Failed to retrieve PDF link from {url}: {e}")
        return None

# List of domains to download from, specifically handling arXiv here
allowed_domains = ['arxiv.org']

# Filter the dataframe for arXiv URLs
filtered_df = Scrape_df[Scrape_df['Base_Domain'].isin(allowed_domains)]

# Iterate over the arXiv URLs and attempt to download the PDFs
for index, row in filtered_df.iterrows():
    url = row['URL']
    
    try:
        # Get the direct PDF link for arXiv
        pdf_url = get_arxiv_pdf_link(url)
        
        if pdf_url:
            # Make a request to download the PDF
            response = requests.get(pdf_url, stream=True)
            
            # Check if the response header indicates it's a PDF
            if response.headers.get('content-type') == 'application/pdf':
                # Construct the filename using the index or title
                filename = f"arxiv_paper_{index}.pdf"
                filepath = os.path.join(output_dir, filename)
                
                # Save the PDF to the output directory
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                print(f"Downloaded: {filename}")
            else:
                print(f"URL does not point to a PDF: {pdf_url}")
        else:
            print(f"Could not find PDF link for {url}")
    
    except Exception as e:
        print(f"Failed to download from {url}: {e}")
'''

Downloaded: arxiv_paper_6.pdf
Downloaded: arxiv_paper_7.pdf
Downloaded: arxiv_paper_24.pdf
Downloaded: arxiv_paper_26.pdf
Downloaded: arxiv_paper_29.pdf
Downloaded: arxiv_paper_51.pdf
Downloaded: arxiv_paper_53.pdf
Downloaded: arxiv_paper_55.pdf
Downloaded: arxiv_paper_56.pdf
Downloaded: arxiv_paper_62.pdf
Downloaded: arxiv_paper_63.pdf
Downloaded: arxiv_paper_69.pdf
Downloaded: arxiv_paper_70.pdf
Downloaded: arxiv_paper_72.pdf
Downloaded: arxiv_paper_75.pdf
Downloaded: arxiv_paper_80.pdf
Downloaded: arxiv_paper_89.pdf
Downloaded: arxiv_paper_96.pdf
Downloaded: arxiv_paper_97.pdf
Downloaded: arxiv_paper_100.pdf
Downloaded: arxiv_paper_106.pdf
Downloaded: arxiv_paper_112.pdf
Downloaded: arxiv_paper_121.pdf
Downloaded: arxiv_paper_128.pdf
Downloaded: arxiv_paper_144.pdf
Downloaded: arxiv_paper_145.pdf
Downloaded: arxiv_paper_170.pdf
Downloaded: arxiv_paper_172.pdf
Downloaded: arxiv_paper_178.pdf
Downloaded: arxiv_paper_187.pdf
Downloaded: arxiv_paper_200.pdf
Downloaded: arxiv_paper_212.p

In [28]:
#pip install PyDF2

In [36]:
import os
import pdfplumber

# Directory where the PDFs are stored
pdf_dir = 'pdfs'

# Function to extract text from a PDF file using pdfplumber
def extract_text_from_pdf(pdf_file):
    try:
        with pdfplumber.open(pdf_file) as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text()
            return text
    except Exception as e:
        print(f"Failed to extract text from {pdf_file}: {e}")
        return ""

# Create a list to store the full text for each paper
full_texts = []

# Iterate over the PDF files in the pdfs directory
for index, row in Scrape_df.iterrows():
    # Construct the PDF file path using index or appropriate identifier
    pdf_file = os.path.join(pdf_dir, f"arxiv_paper_{index}.pdf")
    
    if os.path.exists(pdf_file):
        # Extract text from the PDF
        full_text = extract_text_from_pdf(pdf_file)
        print(f'Full text extracted for index {index}: {bool(full_text)}')  # Verify text extraction
        full_texts.append(full_text)
    else:
        # If the PDF file doesn't exist, append an empty string or None
        full_texts.append("")

# Ensure the length of full_texts matches the number of rows in the dataframe
print(f"Total rows in Scrape_df: {len(Scrape_df)}")
print(f"Total extracted texts: {len(full_texts)}")

# Verify that the lengths match before assigning to the dataframe
if len(full_texts) == len(Scrape_df):
    # Add the extracted full texts to the dataframe
    Scrape_df['Full_Text'] = full_texts
    print("Full text successfully added to the dataframe.")
else:
    print("Error: Mismatch in length between full_texts and dataframe.")

# Display the dataframe to verify
Scrape_df.head()


Full text extracted for index 6: True
Full text extracted for index 7: True
Full text extracted for index 24: True
Full text extracted for index 26: True
Full text extracted for index 29: True
Full text extracted for index 51: True
Full text extracted for index 53: True
Full text extracted for index 55: True
Full text extracted for index 56: True
Full text extracted for index 62: True
Full text extracted for index 63: True
Full text extracted for index 69: True
Full text extracted for index 70: True
Full text extracted for index 72: True
Full text extracted for index 75: True
Full text extracted for index 80: True
Full text extracted for index 89: True
Full text extracted for index 96: True
Full text extracted for index 97: True
Full text extracted for index 100: True
Full text extracted for index 106: True
Full text extracted for index 112: True
Full text extracted for index 121: True
Full text extracted for index 128: True
Full text extracted for index 144: True
Full text extracted f

,Unnamed: 0,Title,Authors,Year,Cited By,Detected_Dataset,Detected_Topic,Abstract,DOI,Journal,...,Mentions_T-test,Mentions_Anova,Mentions_Correlation,Mentions_Regression,Mentions_Baseline_comparison,Mentions_Mae,Mentions_Rmse,Mentions_Bias,Base_Domain,Full_Text
0,0,10 Automated Face Analysis for Affective Compu...,"['JF Cohn', 'F De la Torre']",2015,170,Affective Faces Database,classifier,Differences in manual coding between databases...,No DOI,The Oxford handbook of affective …,...,False,False,False,False,False,False,False,False,oup.com,
1,1,3D facial expression recognition based on auto...,"['H Tang', 'TS Huang']",2008,205,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",facial expression recognition from 3D facial s...,No DOI,… on computer vision and pattern recognition …,...,False,False,False,False,False,False,False,False,ieee.org,
2,2,3D facial expression recognition based on prim...,"['J Wang', 'L Yin', 'X Wei', 'Y Sun']",2006,440,Binghamton University 3D Facial Expression,facial expression recognition,expressions using 3D facial expression range d...,No DOI,… Vision and Pattern Recognition …,...,False,False,False,False,False,False,False,False,ieee.org,
3,3,3D facial expression recognition based on prop...,"['H Tang', 'TS Huang']",2008,153,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",Binghamton University have recently constructe...,No DOI,… on Automatic Face & Gesture Recognition,...,False,False,False,False,False,False,False,False,ieee.org,
4,4,3D facial expression recognition using SIFT de...,"['S Berretti', 'B Ben Amor', 'M Daoudi', 'A De...",2011,184,Binghamton University 3D Facial Expression,"classification, classifier, facial expression ...",at the Binghamton University (BU-3DFE database...,No DOI,The Visual Computer,...,False,False,False,False,False,False,False,False,researchgate.net,


In [40]:
Scrape_df.to_csv('../results/arxiv_full_texts.csv')

# 